In [3]:
# -*- coding: utf-8 -*-
from __future__ import print_function
from __future__ import absolute_import

import sys
import numpy as np
import cv2
import os
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont
import json
import collections
import glob
import pickle
import random

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
def crop_image(img):
    image = np.array(img)
    blur = cv2.GaussianBlur(image, ksize=(3,3), sigmaX=0)
    ret, thresh1 = cv2.threshold(blur, 127, 255, cv2.THRESH_BINARY)
    edged = cv2.Canny(blur, 10, 250)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (7,7))
    closed = cv2.morphologyEx(edged, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(closed.copy(),cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    total = 0
    contours_xy = np.array(contours)
    x_min, x_max = 0,0
    value = list()
    for i in range(len(contours_xy)):
      for j in range(len(contours_xy[i])):
        value.append(contours_xy[i][j][0][0])
        x_min = min(value)
        x_max = max(value)
 
    y_min, y_max = 0,0
    value = list()
    for i in range(len(contours_xy)):
      for j in range(len(contours_xy[i])):
        value.append(contours_xy[i][j][0][1])
        y_min = min(value)
        y_max = max(value)

    x = x_min
    y = y_min
    w = x_max-x_min
    h = y_max-y_min

    return x, y, w, h

def process_image(img, x, y, w, h, canvas_size):
    new_width = int(canvas_size * 0.9)
    new_height = int(new_width * h / w)
    if new_height > canvas_size - 10:
        new_height = int(canvas_size * 0.8)
        new_width = int(new_height * w / h)
    img = img.crop((x-1, y-1, x+w+1, y+h+1)).resize((new_width,new_height))
    new_left = int((canvas_size - img.width) / 2)
    new_top = int((canvas_size - img.height) / 2)
    result = Image.new("L", (canvas_size, canvas_size), color=255)
    result.paste(img, (new_left, new_top))

    return result

In [9]:
def draw_single_char(st, font, canvas_size, final_canvas_size, x_offset, y_offset):
    img = Image.new("RGB", (canvas_size, canvas_size), (255, 255, 255))
    draw = ImageDraw.Draw(img)
    draw.text((x_offset, y_offset), st, (0, 0, 0), font=font)
    x, y, w, h = crop_image(img)
    img = process_image(img, x, y, w, h, final_canvas_size)
    return img


def draw_example(ch, src_font, dst_font, canvas_size, final_canvas_size, x_offset, y_offset):
    dst_img = draw_single_char(ch, dst_font, canvas_size, final_canvas_size, x_offset, y_offset)
    src_img = draw_single_char(ch, src_font, canvas_size, final_canvas_size, x_offset, y_offset)
    example_img = Image.new("RGB", (final_canvas_size * 2, final_canvas_size), (255, 255, 255))
    example_img.paste(dst_img, (0, 0))
    example_img.paste(src_img, (final_canvas_size, 0))
    return example_img


def font2img(src, dst, stringset, char_size, canvas_size, final_canvas_size, x_offset, y_offset, sample_dir, label):
    src_font = ImageFont.truetype(src, size=char_size)
    dst_font = ImageFont.truetype(dst, size=char_size)

    count = 0

    for s in stringset:
        e = draw_example(s, src_font, dst_font, canvas_size, final_canvas_size, x_offset, y_offset)
        if e:
            e.save(os.path.join(sample_dir, "%02d_%04d.jpg" % (label, count)))
            count += 1

In [11]:
src_font = "./gdrive/MyDrive/englishwords/arial.ttf"
path = './gdrive/MyDrive/englishwords/font/'
dst_fonts = os.listdir(path)
f = open("./gdrive/MyDrive/englishwords/student_name.txt", 'r')
string_set = f.readlines()
char_size, canvas_size, final_canvas_size = 100, 600, 128
x_offset, y_offset = 10, 10
sample_dir = './samples/'
shuffle = False
if not os.path.isdir(sample_dir):
  os.mkdir(sample_dir)
label = 1

if shuffle:
    np.random.shuffle(string_set)

for dst_font in dst_fonts[:]:
    font2img(src_font, path + dst_font, string_set, char_size, canvas_size, final_canvas_size, x_offset, y_offset, sample_dir, label)
    print('font#%02d 데이터셋 생성 완료' % (label))
    label += 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  # Remove the CWD from sys.path while we load stuff.


font#01 데이터셋 생성 완료
font#02 데이터셋 생성 완료
font#03 데이터셋 생성 완료
font#04 데이터셋 생성 완료
font#05 데이터셋 생성 완료
font#06 데이터셋 생성 완료
font#07 데이터셋 생성 완료
font#08 데이터셋 생성 완료
font#09 데이터셋 생성 완료
font#10 데이터셋 생성 완료
font#11 데이터셋 생성 완료
font#12 데이터셋 생성 완료
font#13 데이터셋 생성 완료
font#14 데이터셋 생성 완료
font#15 데이터셋 생성 완료
font#16 데이터셋 생성 완료
font#17 데이터셋 생성 완료
font#18 데이터셋 생성 완료
font#19 데이터셋 생성 완료
font#20 데이터셋 생성 완료
font#21 데이터셋 생성 완료
font#22 데이터셋 생성 완료
font#23 데이터셋 생성 완료
font#24 데이터셋 생성 완료
font#25 데이터셋 생성 완료
font#26 데이터셋 생성 완료
font#27 데이터셋 생성 완료
font#28 데이터셋 생성 완료
font#29 데이터셋 생성 완료
font#30 데이터셋 생성 완료
font#31 데이터셋 생성 완료
font#32 데이터셋 생성 완료
font#33 데이터셋 생성 완료
font#34 데이터셋 생성 완료
font#35 데이터셋 생성 완료
font#36 데이터셋 생성 완료
font#37 데이터셋 생성 완료
font#38 데이터셋 생성 완료
font#39 데이터셋 생성 완료
font#40 데이터셋 생성 완료
font#41 데이터셋 생성 완료
font#42 데이터셋 생성 완료
font#43 데이터셋 생성 완료
font#44 데이터셋 생성 완료
font#45 데이터셋 생성 완료
font#46 데이터셋 생성 완료
font#47 데이터셋 생성 완료


In [13]:
def pickle_examples(paths, train_path, val_path, train_val_split):
    with open(train_path, 'wb') as ft:
        with open(val_path, 'wb') as fv:
            for p in paths:
                label = int(os.path.basename(p).split("_")[0])
                with open(p, 'rb') as f:
                    print("img %s" % p, label)
                    img_bytes = f.read()
                    r = random.random()
                    example = (label, img_bytes)
                    if r < train_val_split:
                        pickle.dump(example, fv)
                    else:
                        pickle.dump(example, ft)

save_dir = './gdrive/MyDrive/englishwords/'
split_ratio = 0.1

train_path = os.path.join(save_dir, "train.obj")
val_path = os.path.join(save_dir, "val.obj")
pickle_examples(sorted(glob.glob(os.path.join(sample_dir, "*.jpg"))), train_path=train_path, val_path=val_path, train_val_split=split_ratio)

img ./samples/01_0000.jpg 1
img ./samples/01_0001.jpg 1
img ./samples/01_0002.jpg 1
img ./samples/01_0003.jpg 1
img ./samples/01_0004.jpg 1
img ./samples/01_0005.jpg 1
img ./samples/01_0006.jpg 1
img ./samples/01_0007.jpg 1
img ./samples/01_0008.jpg 1
img ./samples/01_0009.jpg 1
img ./samples/01_0010.jpg 1
img ./samples/01_0011.jpg 1
img ./samples/01_0012.jpg 1
img ./samples/01_0013.jpg 1
img ./samples/01_0014.jpg 1
img ./samples/01_0015.jpg 1
img ./samples/01_0016.jpg 1
img ./samples/01_0017.jpg 1
img ./samples/01_0018.jpg 1
img ./samples/01_0019.jpg 1
img ./samples/01_0020.jpg 1
img ./samples/01_0021.jpg 1
img ./samples/01_0022.jpg 1
img ./samples/01_0023.jpg 1
img ./samples/01_0024.jpg 1
img ./samples/01_0025.jpg 1
img ./samples/01_0026.jpg 1
img ./samples/01_0027.jpg 1
img ./samples/01_0028.jpg 1
img ./samples/01_0029.jpg 1
img ./samples/01_0030.jpg 1
img ./samples/01_0031.jpg 1
img ./samples/01_0032.jpg 1
img ./samples/02_0000.jpg 2
img ./samples/02_0001.jpg 2
img ./samples/02_000